In [ ]:
import sys
sys.path.append('../../Common/')

import CommonMT5

In [ ]:
def detectSignal(symbol, from_date, to_date, timeframe):

    import pandas as pd
    import plotly.graph_objects as go
    import redis
    import numpy as np
    import ta
    from datetime import datetime

    # ##############################################Step 1: Lấy dữ liệu##############################################
    data = CommonMT5.CommonMT5.loaddataMT5_FromTo(symbol, from_date, to_date, timeframe)
    
    # ##############################################Step 2: Chiến lược##############################################  
    # Tính toán các chỉ báo kỹ thuật
    data['MACD'] = ta.trend.MACD(data['Close']).macd_diff()
    data['Stoch'] = ta.momentum.StochasticOscillator(data['High'], data['Low'], data['Close']).stoch()
    data['RSI'] = ta.momentum.RSIIndicator(data['Close']).rsi()
    data['BB_perc'] = (data['Close'] - ta.volatility.BollingerBands(data['Close']).bollinger_lband()) / \
                    (ta.volatility.BollingerBands(data['Close']).bollinger_hband() - ta.volatility.BollingerBands(data['Close']).bollinger_lband())

    # Định nghĩa hàm để kiểm tra điều kiện mua hoặc bán
    def buy_signal(row):
        buy_signals = 0

        if row['MACD'] > 0:
            buy_signals += 1
        if row['Stoch'] < 20:
            buy_signals += 1
        if row['RSI'] < 30:
            buy_signals += 1
        if row['BB_perc'] < 0:
            buy_signals += 1

        return buy_signals >= 1

    def sell_signal(row):
        sell_signals = 0

        if row['MACD'] < 0:
            sell_signals += 1
        if row['Stoch'] > 80:
            sell_signals += 1
        if row['RSI'] > 70:
            sell_signals += 1
        if row['BB_perc'] > 1:
            sell_signals += 1

        return sell_signals >= 1

    # Áp dụng hàm trên dữ liệu
    data['Buy_Signal'] = data.apply(buy_signal, axis=1)
    data['Sell_Signal'] = data.apply(sell_signal, axis=1)
    # data.to_csv('OG_4 Chi Bao.csv')
    # ##############################################Step 3: Đẩy dữ liệu qua Redis##############################################
    # Nếu có tín hiệu thì đẩy qua Redis
    # Datetime  Open    High    Low	Close   Volume  SMA short_ema   long_ema    MACD    Signal_Line Buy_Signal  Sell_Signal
    # Tạo kết nối Redis
    r = redis.Redis(host='localhost', port=6379, db=0)
    # Tạo hash key
    hash_key = 'OG_4 Chi Bao'
    last_record = data.iloc[-1]
   
    # Chuyển đổi record cuối cùng thành từ điển và lưu vào Redis dưới dạng hash
    if(last_record['Buy_Signal'] == True or last_record['Sell_Signal'] == True):
        for field, value in last_record.to_dict().items():
            # Chuyển đổi giá trị uint64 và Timestamp thành chuỗi
            if isinstance(value, pd.Timestamp):
                value = value.isoformat()
            elif isinstance(value, (int, np.uint64)):
                value = str(value)
            r.hset(hash_key, field, value)  
            r.hset(hash_key, 'Symbol', symbol)
            r.hset(hash_key, 'Insertdate', datetime.now().isoformat())
        print(last_record)   
    else:
        print('Không có tín hiệu!')
    # ##############################################Step 4: Vẽ biểu đồ##############################################  
    # data.set_index('Datetime', inplace=True)
    # # Tạo figure
    # fig = go.Figure()
    # # Biểu đồ giá đóng cửa và SMA
    # fig.add_trace(go.Scatter(x=data.index, y=data['Close'], mode='lines', name='Giá Đóng Cửa', line=dict(color='blue')))
    # fig.add_trace(go.Scatter(x=data.index, y=data['SMA'], mode='lines', name='SMA', line=dict(color='orange')))
    # # Vẽ các điểm cho tín hiệu mua
    # buy_signals = data[data['Buy_Signal']]
    # fig.add_trace(go.Scatter(x=buy_signals.index, y=buy_signals['Close'], mode='markers', name='Buy Signal', marker=dict(symbol='triangle-up', size=10, color='green')))
    # # Vẽ các điểm cho tín hiệu bán
    # sell_signals = data[data['Sell_Signal']]
    # fig.add_trace(go.Scatter(x=sell_signals.index, y=sell_signals['Close'], mode='markers', name='Sell Signal', marker=dict(symbol='triangle-down', size=10, color='red')))
    # fig.update_layout(title='Giá Đóng Cửa và SMA', yaxis_title='Giá', xaxis_rangeslider_visible=False, height=500)
    # # Hiển thị biểu đồ
    # fig.show()

    # ####################################################################################################
    # # Tạo figure
    # fig = go.Figure()
    # # Biểu đồ MACD và Signal Line
    # fig.add_trace(go.Scatter(x=data.index, y=data['MACD'], mode='lines', name='MACD', line=dict(color='blue')))
    # fig.add_trace(go.Scatter(x=data.index, y=data['Signal_Line'], mode='lines', name='Signal Line', line=dict(color='orange')))
    # fig.update_layout(title='MACD và Signal Line', yaxis_title='MACD', xaxis_rangeslider_visible=False, height=500)
    # # Hiển thị biểu đồ
    # fig.show()

In [3]:
import MetaTrader5 as mt5
from datetime import datetime, timedelta
import ta
import schedule
import time

def schedule_detectSignal(timeframe):
    symbol = 'XAUUSDc'
    from_date = (datetime.now() - timedelta(days=5)).strftime('%Y-%m-%d')  # Lấy ngày hôm qua
    to_date = (datetime.now() + timedelta(days=1)).strftime('%Y-%m-%d')
    timeframe = mt5.TIMEFRAME_M1 # 1m
    # 0x4000
    detectSignal(symbol, from_date, to_date, timeframe)

# Danh sách các phút cụ thể bạn muốn hàm được chạy
run_minutes = list(range(0,60,1))
while True:
    current_time = datetime.now()
    current_minute = current_time.minute
    # Kiểm tra xem phút hiện tại có nằm trong danh sách các phút cần chạy hàm không
    if current_minute in run_minutes:
        # Kiểm tra xem đã chạy hàm trong phút hiện tại hay chưa
        last_run = getattr(schedule_detectSignal, 'last_run', None)
        if last_run is None or last_run != current_minute:
            schedule_detectSignal(mt5.TIMEFRAME_M1)
            # Lưu lại phút cuối cùng mà hàm đã chạy
            setattr(schedule_detectSignal, 'last_run', current_minute)

    # Nghỉ 1 giây trước khi kiểm tra lại
    time.sleep(1)

Datetime       2024-02-01 17:00:00
Open                       2045.92
High                       2045.92
Low                        2040.45
Close                      2044.53
Volume                         446
MACD                     -0.483687
Stoch                    42.367601
RSI                      33.709819
BB_perc                  -0.240334
Buy_Signal                    True
Sell_Signal                   True
Name: 5493, dtype: object
Datetime       2024-02-01 17:00:00
Open                       2045.92
High                       2045.92
Low                        2040.45
Close                      2044.53
Volume                         446
MACD                     -0.483687
Stoch                    42.367601
RSI                      33.709819
BB_perc                  -0.240334
Buy_Signal                    True
Sell_Signal                   True
Name: 5493, dtype: object
Datetime       2024-02-01 17:00:00
Open                       2045.92
High                       2045.92
Low